# 1. Packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import QuantileRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
import joblib
from sklearn.preprocessing import LabelEncoder
import pickle
import ml_utils as mlu

# Reading data

In [ ]:
train = pd.read_csv("use_to_train.csv")
test = pd.read_csv("use_to_test.csv")
validation = pd.read_csv("use_to_val.csv")

In [ ]:
# Select variables
# feature_names_number = ['fuel_consumption_km_l', 'horsepower', 'displacement', 'gears', 'torque', 'passengers', 'doors', 'wheels', 'km', 'age']
# feature_names_category = ['touch_screen', 'navigation_system','rear_sensor',  'start_stop', 'turbo', 'seat_material', 'trunk_opening',  'body_type', 'electric_parking_brake', 'electric_locks']

# feature_names_number = ['age', 'km', 'fuel_consumption_km_l', 'horsepower', 'displacement', 'torque']
# feature_names_category = ['push_start','start_stop', 'turbo', 'electric_locks', 'navigation_system']

feature_names_number = train.select_dtypes(include='number').columns.tolist()
feature_names_number.remove('price')
feature_names_category = train.select_dtypes(include='object').columns.tolist()

print('feature_names_number')
print(feature_names_number)
print('feature_names_category')
print(feature_names_category)

feature_names_number
['fuel_consumption_km_l', 'horsepower', 'displacement', 'gears', 'torque', 'passengers', 'doors', 'wheels', 'km', 'age']
feature_names_category
['touch_screen', 'navigation_system', 'push_start', 'rear_cupholders', 'sunroof', 'rear_sensor', 'start_stop', 'turbo', 'seat_material', 'trunk_opening', 'headlights', 'body_type', 'electric_parking_brake', 'electric_locks']


# 2. Partition of data

In [ ]:
X_train = train[feature_names_number + feature_names_category]
y_train = train['price']
X_test = test[feature_names_number + feature_names_category]
y_test = test['price']
X_val = validation[feature_names_number + feature_names_category]
y_val = validation['price']

# 3. Preprocessing

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

In [ ]:
attributes_number = Pipeline(steps=[
    ('null_replacement', SimpleImputer(strategy='mean')),
    ('scaling', StandardScaler())
])

attributes_category = Pipeline(steps=[
    ('null_replacement', SimpleImputer(strategy='most_frequent')),
    ('encoding', OneHotEncoder(handle_unknown='ignore',sparse_output=False))
])

attributes_preprocessing = ColumnTransformer(transformers = [
    ('number', attributes_number, feature_names_number),
    ('category', attributes_category, feature_names_category)    
])

# 4. Evaluate Model

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
def get_metrics_pd(y_values, preds, model_name):
    mse = np.round(mean_squared_error(y_values, preds),3)
    r2 = np.round(r2_score(y_values, preds),3)
    mae = np.round(mean_absolute_error(y_values, preds),3)
    rmse = np.round(np.sqrt(mse),3)
    return pd.Series([mse, r2, mae, rmse], index=['MSE', 'R^2', 'MAE', 'RMSE'])

# 5. Hyper parameters 

In [ ]:
# !pip install catboost
# !pip install optuna

# Extreme

In [ ]:
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, ElasticNet, 
                                  HuberRegressor, BayesianRidge, TheilSenRegressor, 
                                  RANSACRegressor, PassiveAggressiveRegressor, 
                                  OrthogonalMatchingPursuit)
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet(),
    'SVR': SVR(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'AdaBoostRegressor': AdaBoostRegressor(),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'XGBRegressor': XGBRegressor(),
    'LGBMRegressor': LGBMRegressor(),
    'CatBoostRegressor': CatBoostRegressor(),
    'HuberRegressor': HuberRegressor(),
    'BayesianRidge': BayesianRidge(),
    'TheilSenRegressor': TheilSenRegressor(),
    'RANSACRegressor': RANSACRegressor(),
    'PassiveAggressiveRegressor': PassiveAggressiveRegressor(),
    'OrthogonalMatchingPursuit': OrthogonalMatchingPursuit()
}

In [ ]:
%%time
df_train_metrics = pd.DataFrame()
df_test_metrics = pd.DataFrame()
df_val_metrics = pd.DataFrame()
for name, model in models.items():
    print(f'Training {name}')
    model_pipeline = Pipeline(steps=[
        ('preprocess', attributes_preprocessing),
        ('model', model)
    ])

    model_pipeline.fit(X_train, y_train)
    
    print("Getting train metrics:")
    train_metrics = mlu.get_metrics_pd(y_train, model_pipeline.predict(X_train), name)
    df_train_metrics = pd.concat([df_train_metrics, train_metrics], axis=1)

    print("Getting test metrics:")
    test_metrics = mlu.get_metrics_pd(y_test, model_pipeline.predict(X_test), name)
    df_test_metrics = pd.concat([df_test_metrics, test_metrics], axis=1)

    print("Getting validation metrics:")
    val_metrics = mlu.get_metrics_pd(y_val, model_pipeline.predict(X_val), name)
    df_val_metrics = pd.concat([df_val_metrics, val_metrics], axis=1)

38:	learn: 63427.0371855	total: 123ms	remaining: 3.04s
39:	learn: 62668.8038299	total: 125ms	remaining: 3s
40:	learn: 62027.9178576	total: 127ms	remaining: 2.97s
41:	learn: 61290.3367467	total: 129ms	remaining: 2.93s
42:	learn: 60553.5787925	total: 130ms	remaining: 2.9s
43:	learn: 59909.1836764	total: 132ms	remaining: 2.87s
44:	learn: 59290.3831996	total: 134ms	remaining: 2.84s
45:	learn: 58706.9071487	total: 135ms	remaining: 2.81s
46:	learn: 58127.5409602	total: 137ms	remaining: 2.78s
47:	learn: 57547.5992487	total: 139ms	remaining: 2.75s
48:	learn: 56906.9685913	total: 140ms	remaining: 2.73s
49:	learn: 56376.0592397	total: 142ms	remaining: 2.7s
50:	learn: 55930.6369509	total: 144ms	remaining: 2.68s
51:	learn: 55393.6558395	total: 146ms	remaining: 2.65s
52:	learn: 54850.0018488	total: 147ms	remaining: 2.63s
53:	learn: 54372.0519059	total: 149ms	remaining: 2.61s
54:	learn: 53826.8784103	total: 151ms	remaining: 2.59s
55:	learn: 53341.9114409	total: 152ms	remaining: 2.57s
56:	learn: 5288

## Training metrics

In [ ]:
df_train_metrics = df_train_metrics.T.sort_values(by='RMSE')
# df_train_metrics.index.name = 'Model'
# df_train_metrics.reset_index(inplace=True)
df_train_metrics

,MSE,R^2,MAE,RMSE
DecisionTreeRegressor,6.275347e+05,1.000,132.337,792.171
XGBRegressor,7.659561e+06,1.000,1975.037,2767.591
CatBoostRegressor,1.612989e+08,0.990,9543.504,12700.350
RandomForestRegressor,3.323253e+08,0.979,11695.288,18229.792
LGBMRegressor,4.231509e+08,0.973,12219.654,20570.631
GradientBoostingRegressor,8.411269e+08,0.946,21351.745,29002.187
KNeighborsRegressor,2.355314e+09,0.848,31737.247,48531.580
LinearRegression,2.814722e+09,0.819,37750.962,53053.956
Lasso,2.814724e+09,0.819,37750.002,53053.974
Ridge,2.817439e+09,0.819,37782.349,53079.552


## Test metrics

In [ ]:
df_test_metrics = df_test_metrics.T.sort_values(by='RMSE')
# df_test_metrics = df_test_metrics.index.name = 'Model'
# df_test_metrics.reset_index(inplace=True)
df_test_metrics

,MSE,R^2,MAE,RMSE
CatBoostRegressor,2.027566e+09,0.897,27004.779,45028.504
GradientBoostingRegressor,2.485331e+09,0.873,31524.158,49853.097
LGBMRegressor,2.490445e+09,0.873,30367.526,49904.361
XGBRegressor,2.677129e+09,0.863,30892.098,51740.983
RandomForestRegressor,2.823976e+09,0.856,32882.582,53141.098
BayesianRidge,4.097034e+09,0.791,44108.849,64008.080
Ridge,4.140021e+09,0.789,44182.956,64342.992
Lasso,4.161280e+09,0.788,44197.938,64507.980
LinearRegression,4.162009e+09,0.788,44200.637,64513.634
TheilSenRegressor,4.298016e+09,0.781,43647.185,65559.253


## Validation metrics

In [ ]:
df_val_metrics = df_val_metrics.T.sort_values(by='RMSE')
# df_val_metrics.index.name = 'Model'
# df_val_metrics.reset_index(inplace=True)
df_val_metrics

,MSE,R^2,MAE,RMSE
CatBoostRegressor,1.595510e+09,0.887,26071.341,39943.840
XGBRegressor,2.159432e+09,0.847,28953.559,46469.685
RandomForestRegressor,2.182417e+09,0.845,30221.749,46716.342
LGBMRegressor,2.282189e+09,0.838,29494.251,47772.263
GradientBoostingRegressor,2.368786e+09,0.832,31509.790,48670.176
HuberRegressor,2.969031e+09,0.789,37003.684,54488.813
TheilSenRegressor,2.974646e+09,0.789,38700.989,54540.318
PassiveAggressiveRegressor,3.030075e+09,0.785,37304.428,55046.115
BayesianRidge,3.049265e+09,0.784,40075.444,55220.148
Ridge,3.118013e+09,0.779,40356.276,55839.169


In [ ]:
df_full_metris = pd.concat([df_train_metrics, df_test_metrics, df_val_metrics], axis=1)
df_full_metris.index.name = 'Model'
df_full_metris.reset_index(inplace=True)
df_full_metris

,Model,MSE,R^2,MAE,RMSE,MSE,R^2,MAE,RMSE,MSE,R^2,MAE,RMSE
0,DecisionTreeRegressor,6.275347e+05,1.000,132.337,792.171,6.716132e+09,0.657,46840.812,81952.010,4.661619e+09,0.669,43318.763,68276.049
1,XGBRegressor,7.659561e+06,1.000,1975.037,2767.591,2.677129e+09,0.863,30892.098,51740.983,2.159432e+09,0.847,28953.559,46469.685
2,CatBoostRegressor,1.612989e+08,0.990,9543.504,12700.350,2.027566e+09,0.897,27004.779,45028.504,1.595510e+09,0.887,26071.341,39943.840
3,RandomForestRegressor,3.323253e+08,0.979,11695.288,18229.792,2.823976e+09,0.856,32882.582,53141.098,2.182417e+09,0.845,30221.749,46716.342
4,LGBMRegressor,4.231509e+08,0.973,12219.654,20570.631,2.490445e+09,0.873,30367.526,49904.361,2.282189e+09,0.838,29494.251,47772.263
5,GradientBoostingRegressor,8.411269e+08,0.946,21351.745,29002.187,2.485331e+09,0.873,31524.158,49853.097,2.368786e+09,0.832,31509.790,48670.176
6,KNeighborsRegressor,2.355314e+09,0.848,31737.247,48531.580,4.786587e+09,0.756,44032.479,69185.162,3.266764e+09,0.768,38078.038,57155.616
7,LinearRegression,2.814722e+09,0.819,37750.962,53053.956,4.162009e+09,0.788,44200.637,64513.634,3.146115e+09,0.777,40518.071,56090.241
8,Lasso,2.814724e+09,0.819,37750.002,53053.974,4.161280e+09,0.788,44197.938,64507.980,3.145392e+09,0.777,40513.426,56083.796
9,Ridge,2.817439e+09,0.819,37782.349,53079.552,4.140021e+09,0.789,44182.956,64342.992,3.118013e+09,0.779,40356.276,55839.169


In [ ]:
df_full_metris.T


,DecisionTreeRegressor,XGBRegressor,CatBoostRegressor,RandomForestRegressor,LGBMRegressor,GradientBoostingRegressor,KNeighborsRegressor,LinearRegression,Lasso,Ridge,BayesianRidge,TheilSenRegressor,HuberRegressor,PassiveAggressiveRegressor,RANSACRegressor,OrthogonalMatchingPursuit,AdaBoostRegressor,ElasticNet,SVR
MSE,6.275347e+05,7.659561e+06,1.612989e+08,3.323253e+08,4.231509e+08,8.411269e+08,2.355314e+09,2.814722e+09,2.814724e+09,2.817439e+09,2.841880e+09,2.975687e+09,3.155678e+09,3.496532e+09,3.631866e+09,3.759143e+09,3.827813e+09,3.985817e+09,1.639257e+10
R^2,1.000000e+00,1.000000e+00,9.900000e-01,9.790000e-01,9.730000e-01,9.460000e-01,8.480000e-01,8.190000e-01,8.190000e-01,8.190000e-01,8.170000e-01,8.090000e-01,7.970000e-01,7.750000e-01,7.660000e-01,7.580000e-01,7.540000e-01,7.440000e-01,-5.500000e-02
MAE,1.323370e+02,1.975037e+03,9.543504e+03,1.169529e+04,1.221965e+04,2.135174e+04,3.173725e+04,3.775096e+04,3.775000e+04,3.778235e+04,3.806206e+04,3.746193e+04,3.548786e+04,3.756158e+04,3.767683e+04,4.395808e+04,5.284259e+04,4.297404e+04,9.274144e+04
RMSE,7.921710e+02,2.767591e+03,1.270035e+04,1.822979e+04,2.057063e+04,2.900219e+04,4.853158e+04,5.305396e+04,5.305397e+04,5.307955e+04,5.330929e+04,5.454986e+04,5.617542e+04,5.913148e+04,6.026496e+04,6.131185e+04,6.186932e+04,6.313333e+04,1.280335e+05
MSE,6.716132e+09,2.677129e+09,2.027566e+09,2.823976e+09,2.490445e+09,2.485331e+09,4.786587e+09,4.162009e+09,4.161280e+09,4.140021e+09,4.097034e+09,4.298016e+09,4.620665e+09,4.688456e+09,5.158664e+09,4.870913e+09,5.307495e+09,5.130688e+09,2.107344e+10
R^2,6.570000e-01,8.630000e-01,8.970000e-01,8.560000e-01,8.730000e-01,8.730000e-01,7.560000e-01,7.880000e-01,7.880000e-01,7.890000e-01,7.910000e-01,7.810000e-01,7.640000e-01,7.610000e-01,7.370000e-01,7.520000e-01,7.290000e-01,7.380000e-01,-7.500000e-02
MAE,4.684081e+04,3.089210e+04,2.700478e+04,3.288258e+04,3.036753e+04,3.152416e+04,4.403248e+04,4.420064e+04,4.419794e+04,4.418296e+04,4.410885e+04,4.364718e+04,4.223348e+04,4.297125e+04,4.458315e+04,4.954054e+04,5.843883e+04,4.670045e+04,9.769622e+04
RMSE,8.195201e+04,5.174098e+04,4.502850e+04,5.314110e+04,4.990436e+04,4.985310e+04,6.918516e+04,6.451363e+04,6.450798e+04,6.434299e+04,6.400808e+04,6.555925e+04,6.797548e+04,6.847230e+04,7.182384e+04,6.979193e+04,7.285256e+04,7.162882e+04,1.451669e+05
MSE,4.661619e+09,2.159432e+09,1.595510e+09,2.182417e+09,2.282189e+09,2.368786e+09,3.266764e+09,3.146115e+09,3.145392e+09,3.118013e+09,3.049265e+09,2.974646e+09,2.969031e+09,3.030075e+09,3.338361e+09,3.485567e+09,4.508074e+09,3.311408e+09,1.447140e+10
R^2,6.690000e-01,8.470000e-01,8.870000e-01,8.450000e-01,8.380000e-01,8.320000e-01,7.680000e-01,7.770000e-01,7.770000e-01,7.790000e-01,7.840000e-01,7.890000e-01,7.890000e-01,7.850000e-01,7.630000e-01,7.530000e-01,6.800000e-01,7.650000e-01,-2.700000e-02


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a817877f-bc24-4404-a1bc-b1f4f599a592' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>